## Assignment plan:
- Crawl the whole Wiki web page - `Done`;
- Crawl initial all links that lead to each films's individual web page - `Done`;
- Crawl the data for each film using Scrapy Spider `films_spider.py` as JSON file `films_v2_res.json` - `Done`;
- Clean the data and save it in an appropriate format - `Done`;
- Create a DB for storing all files;
...

In [676]:
# Import necessary dependencies
import scrapy
from urllib.parse import urljoin
from lxml import etree
import pandas as pd
import re

In [677]:
# Primary information source link
web_link = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
main_link = "https://en.wikipedia.org"

In [678]:
# Start a project with Scrapy
#! scrapy startproject wiki_parsing

### Get the links for each film

In [679]:
with open("/home/viper/Data_science/DWV/assignments/assignment_1/wiki_parsing/quotes-wiki.html", "r", encoding="utf-8") as file:
    html_content = file.read()

# Parse the HTML content
tree = etree.HTML(html_content)

table = tree.xpath('//table[contains(@class, "wikitable plainrowheaders")]')[1] # We need the second table that is sorted by year
rows = table.xpath('.//tr')


for row in rows:
    cells = row.xpath('.//td//i//a//@href')
    print(cells)

[]
['/wiki/The_Birth_of_a_Nation']
['/wiki/Intolerance_(film)']
['/wiki/Cleopatra_(1917_film)']
['/wiki/Mickey_(1918_film)']
['/wiki/The_Miracle_Man_(1919_film)']
['/wiki/Way_Down_East']
['/wiki/The_Four_Horsemen_of_the_Apocalypse_(film)']
['/wiki/Douglas_Fairbanks_in_Robin_Hood']
['/wiki/The_Covered_Wagon']
['/wiki/The_Sea_Hawk_(1924_film)']
['/wiki/The_Big_Parade']
['/wiki/Ben-Hur_(1925_film)']
['/wiki/For_Heaven%27s_Sake_(1926_film)']
['/wiki/Wings_(1927_film)']
['/wiki/The_Singing_Fool']
['/wiki/The_Broadway_Melody']
['/wiki/Sunny_Side_Up_(1929_film)']
['/wiki/All_Quiet_on_the_Western_Front_(1930_film)']
['/wiki/Frankenstein_(1931_film)']
['/wiki/City_Lights']
['/wiki/The_Sign_of_the_Cross_(1932_film)']
['/wiki/King_Kong_(1933_film)']
['/wiki/I%27m_No_Angel']
['/wiki/Cavalcade_(1933_film)']
['/wiki/She_Done_Him_Wrong']
['/wiki/The_Merry_Widow_(1934_film)']
['/wiki/It_Happened_One_Night']
['/wiki/Mutiny_on_the_Bounty_(1935_film)']
['/wiki/San_Francisco_(1936_film)']
['/wiki/Snow_Whi

In [680]:
len(rows)

152

### Create a spider to parse all the main information about each film (Saved as `films_spider.py`)

In [681]:
class FilmsSpider(scrapy.Spider):
    name = "films_v2"
    start_urls = [
        "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
    ]

    def parse(self, response):
        # Extract all film links from the second table with films by year
        table = response.xpath('//table[contains(@class, "wikitable plainrowheaders")]')[1]
        rows = table.xpath(".//tr")
        film_links = []
        for row in rows:
            res = row.xpath('.//td//i//a//@href').getall()
            if len(res) == 0:
                continue
            else:
                film_links.append(res[0])

        # Get data from each film link
        for link in film_links:
            absolute_url = urljoin("https://en.wikipedia.org", link)
            yield scrapy.Request(url=absolute_url, callback=self.parse_film_page)

    def parse_film_page(self, response):
        info_table = response.xpath('//table[contains(@class, "infobox")]')
        data = {
            "title": info_table.xpath('.//th[contains(@class, "infobox-above")]//text()').get(),
            "directed_by": info_table.xpath('.//th[contains(text(), "Directed by")]/following-sibling::td//text()').getall(),
            "screenplay_by": info_table.xpath('.//th[contains(text(), "Screenplay by")]/following-sibling::td//text()').getall(),
            "based_on": info_table.xpath('.//th[contains(text(), "Based on")]/following-sibling::td//text()').getall(),
            "produced_by": info_table.xpath('.//th[contains(text(), "Produced by")]/following-sibling::td//text()').getall(),
            "starring": info_table.xpath('.//th[contains(text(), "Starring")]/following-sibling::td//text()').getall(),
            "release_date": info_table.xpath('.//th[contains(text(), "Release date")]/following-sibling::td//text()').get(),
            "country": info_table.xpath('.//th[contains(text(), "Country")]/following-sibling::td//text()').getall(),
            "budget": info_table.xpath('.//th[contains(text(), "Budget")]/following-sibling::td//text()').getall(),
            "box_office": info_table.xpath('.//th[contains(text(), "Box office")]/following-sibling::td//text()').getall(),
        }

        yield data

### Read the data from JSON file and clean it

In [682]:
df = pd.read_json("wiki_parsing/films_v2_res.json")
df

,title,directed_by,screenplay_by,based_on,produced_by,starring,release_date,country,budget,box_office
0,The Birth of a Nation,[D. W. Griffith],"[.mw-parser-output .plainlist ol,.mw-parser-ou...","[The Clansman, by , Thomas Dixon Jr.]","[D. W. Griffith, Harry Aitken, [, 1, ]]","[\n, Lillian Gish, \n, Mae Marsh, \n, Henry B....",[],[United States],"[$100,000+, [, 3, ]]","[$50–100 million, [, 4, ]]"
1,The Greatest Show on Earth,[Cecil B. DeMille],"[.mw-parser-output .plainlist ol,.mw-parser-ou...",[],[Cecil B. DeMille],"[Betty Hutton, Cornel Wilde, Charlton Heston, ...",[],[United States],"[$4 million, [, 1, ]]","[$36 million, [, 1, ]]"
2,This Is Cinerama,"[.mw-parser-output .plainlist ol,.mw-parser-ou...",[],[],"[Robert L. Bendick, Merian C. Cooper]",[Lowell Thomas],[],[United States],[$1 million],"[$41.6 million, [, 2, ]]"
3,Cinerama Holiday,"[Robert L. Bendick, Philippe De Lacy]",[],[],[],"[Fred Troller, , Beatrice Troller , John Mars...",[],[United States],"[$1.5 million, [, 1, ]]","[$29.6 million , [, 2, ]]"
4,The Robe,[Henry Koster],"[.mw-parser-output .plainlist ol,.mw-parser-ou...","[The Robe, by , Lloyd C. Douglas]",[Frank Ross],"[\n, Richard Burton, \n, Jean Simmons, \n, Vic...",[],[United States],"[$4.1 million, [, 1, ], – $4.6 million, [, 2, ]]","[$36 million (United States), [, 3, ]]"
...,...,...,...,...,...,...,...,...,...,...
145,Cleopatra,[J. Gordon Edwards],[Adrian Johnson],"[Cleopatra, 1889 novel, by , H. Rider Haggard]",[William Fox],"[Theda Bara, Fritz Leiber Sr., Thurston Hall]",[],[United States],"[$250,000–500,000, [, 1, ]]","[$1 million, [, 1, ]]"
146,The Miracle Man,[George Loane Tucker],[],"[The Miracle Man, (play), by , George M. Cohan]",[George Loane Tucker],"[Thomas Meighan, Betty Compson, Lon Chaney, Jo...",[],[United States],"[$120,000]","[$1 million (rentals), [, 1, ], or $2 million..."
147,The Four Horsemen of the Apocalypse,[Rex Ingram],[June Mathis],"[The Four Horsemen of the Apocalypse, 1916 nov...",[Rex Ingram],"[Pomeroy Cannon, Josef Swickard, Bridgetta Cla...",[],[United States],"[$800,000 or $1 million, [, 1, ]]","[$9.2 million, [, 2, ], or $4 million (world ..."
148,Mickey,"[F. Richard Jones, James Young]",[],[],"[Mabel Normand, Mack Sennett]",[Mabel Normand],[],[United States],"[$250,000]","[$16,450,000]"


In [683]:
df.loc[0, 'screenplay_by']

['.mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding:0}.mw-parser-output .plainlist ol li,.mw-parser-output .plainlist ul li{margin-bottom:0}',
 'D. W. Griffith',
 'Frank E. Woods']

In [684]:
df.loc[:, 'release_date']

0      []
1      []
2      []
3      []
4      []
       ..
145    []
146    []
147    []
148    []
149    []
Name: release_date, Length: 150, dtype: object

As you can see, the most major problem right now - absense of the release date. It may happen due to 2 possible options as:
- Release date - if the film has one release date;
- Release dates - if the film has multiple release dates.

However, we can still parse this data from the inital table.

In [685]:
table = tree.xpath('//table[contains(@class, "wikitable plainrowheaders")]')[1] # We need the second table that is sorted by year
rows = table.xpath('.//tr')

release_years = []
for row in rows:
    cells = row.xpath('.//th//a//text()')
    release_years.append(cells)
    print(cells)

[]
['1915']
['1916']
['1917']
['1918']
['1919']
['1920']
['1921']
['1922']
['1923']
['1924']
['1925']
[]
['1926']
['1927']
['1928']
['1929']
[]
['1930']
['1931']
[]
['1932']
['1933']
[]
[]
[]
['1934']
[]
['1935']
['1936']
['1937']
['1938']
['1939']
['1940']
[]
['1941']
['1942']
[]
['1943']
[]
['1944']
['1945']
[]
['1946']
[]
[]
['1947']
[]
['1948']
[]
[]
['1949']
['1950']
[]
['1951']
['1952']
[]
['1953']
[]
['1954']
[]
[]
['1955']
[]
[]
['1956']
['1957']
['1958']
['1959']
['1960']
[]
[]
['1961']
[]
['1962']
[]
[]
['1963']
[]
['1964']
[]
[]
['1965']
['1966']
[]
[]
['1967']
[]
['1968']
[]
['1969']
['1970']
['1971']
[]
[]
['1972']
['1973']
[]
['1974']
['1975']
['1976']
['1977']
['1978']
['1979']
[]
['1980']
['1981']
['1982']
['1983']
['1984']
['1985']
['1986']
['1987']
['1988']
['1989']
['1990']
['1991']
['1992']
['1993']
['1994']
['1995']
[]
['1996']
['1997']
['1998']
['1999']
['2000']
['2001']
['2002']
['2003']
['2004']
['2005']
['2006']
['2007']
['2008']
['2009']
['2010']
['2011']
['20

In [686]:
# Delete the first empty element
release_years = release_years[1:]

As you can see from the list (and website itself), empty elements means that the previous year was used for the film with empty release date. Let's fix it.

In [687]:
last_release = None
for release_year_idx in range(len(release_years)):
    if len(release_years[release_year_idx]):
        last_release = release_years[release_year_idx]
    else:
        release_years[release_year_idx] = last_release
release_years

[['1915'],
 ['1916'],
 ['1917'],
 ['1918'],
 ['1919'],
 ['1920'],
 ['1921'],
 ['1922'],
 ['1923'],
 ['1924'],
 ['1925'],
 ['1925'],
 ['1926'],
 ['1927'],
 ['1928'],
 ['1929'],
 ['1929'],
 ['1930'],
 ['1931'],
 ['1931'],
 ['1932'],
 ['1933'],
 ['1933'],
 ['1933'],
 ['1933'],
 ['1934'],
 ['1934'],
 ['1935'],
 ['1936'],
 ['1937'],
 ['1938'],
 ['1939'],
 ['1940'],
 ['1940'],
 ['1941'],
 ['1942'],
 ['1942'],
 ['1943'],
 ['1943'],
 ['1944'],
 ['1945'],
 ['1945'],
 ['1946'],
 ['1946'],
 ['1946'],
 ['1947'],
 ['1947'],
 ['1948'],
 ['1948'],
 ['1948'],
 ['1949'],
 ['1950'],
 ['1950'],
 ['1951'],
 ['1952'],
 ['1952'],
 ['1953'],
 ['1953'],
 ['1954'],
 ['1954'],
 ['1954'],
 ['1955'],
 ['1955'],
 ['1955'],
 ['1956'],
 ['1957'],
 ['1958'],
 ['1959'],
 ['1960'],
 ['1960'],
 ['1960'],
 ['1961'],
 ['1961'],
 ['1962'],
 ['1962'],
 ['1962'],
 ['1963'],
 ['1963'],
 ['1964'],
 ['1964'],
 ['1964'],
 ['1965'],
 ['1966'],
 ['1966'],
 ['1966'],
 ['1967'],
 ['1967'],
 ['1968'],
 ['1968'],
 ['1969'],
 ['1970'],

In [688]:
# Remove "Ne Zha 2" from the last element since we have no information about this film
release_years = release_years[:-1]
release_years

[['1915'],
 ['1916'],
 ['1917'],
 ['1918'],
 ['1919'],
 ['1920'],
 ['1921'],
 ['1922'],
 ['1923'],
 ['1924'],
 ['1925'],
 ['1925'],
 ['1926'],
 ['1927'],
 ['1928'],
 ['1929'],
 ['1929'],
 ['1930'],
 ['1931'],
 ['1931'],
 ['1932'],
 ['1933'],
 ['1933'],
 ['1933'],
 ['1933'],
 ['1934'],
 ['1934'],
 ['1935'],
 ['1936'],
 ['1937'],
 ['1938'],
 ['1939'],
 ['1940'],
 ['1940'],
 ['1941'],
 ['1942'],
 ['1942'],
 ['1943'],
 ['1943'],
 ['1944'],
 ['1945'],
 ['1945'],
 ['1946'],
 ['1946'],
 ['1946'],
 ['1947'],
 ['1947'],
 ['1948'],
 ['1948'],
 ['1948'],
 ['1949'],
 ['1950'],
 ['1950'],
 ['1951'],
 ['1952'],
 ['1952'],
 ['1953'],
 ['1953'],
 ['1954'],
 ['1954'],
 ['1954'],
 ['1955'],
 ['1955'],
 ['1955'],
 ['1956'],
 ['1957'],
 ['1958'],
 ['1959'],
 ['1960'],
 ['1960'],
 ['1960'],
 ['1961'],
 ['1961'],
 ['1962'],
 ['1962'],
 ['1962'],
 ['1963'],
 ['1963'],
 ['1964'],
 ['1964'],
 ['1964'],
 ['1965'],
 ['1966'],
 ['1966'],
 ['1966'],
 ['1967'],
 ['1967'],
 ['1968'],
 ['1968'],
 ['1969'],
 ['1970'],

In [689]:
assert len(release_years) == df.shape[0], (
    "Lengths must match."
)

In [690]:
# Add the release years to our df
df['release_date'] = release_years
df.head()

,title,directed_by,screenplay_by,based_on,produced_by,starring,release_date,country,budget,box_office
0,The Birth of a Nation,[D. W. Griffith],"[.mw-parser-output .plainlist ol,.mw-parser-ou...","[The Clansman, by , Thomas Dixon Jr.]","[D. W. Griffith, Harry Aitken, [, 1, ]]","[\n, Lillian Gish, \n, Mae Marsh, \n, Henry B....",[1915],[United States],"[$100,000+, [, 3, ]]","[$50–100 million, [, 4, ]]"
1,The Greatest Show on Earth,[Cecil B. DeMille],"[.mw-parser-output .plainlist ol,.mw-parser-ou...",[],[Cecil B. DeMille],"[Betty Hutton, Cornel Wilde, Charlton Heston, ...",[1916],[United States],"[$4 million, [, 1, ]]","[$36 million, [, 1, ]]"
2,This Is Cinerama,"[.mw-parser-output .plainlist ol,.mw-parser-ou...",[],[],"[Robert L. Bendick, Merian C. Cooper]",[Lowell Thomas],[1917],[United States],[$1 million],"[$41.6 million, [, 2, ]]"
3,Cinerama Holiday,"[Robert L. Bendick, Philippe De Lacy]",[],[],[],"[Fred Troller, , Beatrice Troller , John Mars...",[1918],[United States],"[$1.5 million, [, 1, ]]","[$29.6 million , [, 2, ]]"
4,The Robe,[Henry Koster],"[.mw-parser-output .plainlist ol,.mw-parser-ou...","[The Robe, by , Lloyd C. Douglas]",[Frank Ross],"[\n, Richard Burton, \n, Jean Simmons, \n, Vic...",[1919],[United States],"[$4.1 million, [, 1, ], – $4.6 million, [, 2, ]]","[$36 million (United States), [, 3, ]]"


In [691]:
# Unpack the release_date column
df['release_date'] = df.loc[:, 'release_date'].apply(lambda x: x[0])

Now we have to investigate and preprocess each column & delete those with few information.

In [692]:
# Check if there are any empty titles
df.loc[:, 'title'].apply(lambda x: 1 if len(x) == 0 else 0).sum()

np.int64(0)

In [693]:
# Check if there are any empty directed by entries & remove any incorrect data
df.loc[:, 'directed_by'].apply(lambda x: 1 if len(x) == 0 else 0).sum()

np.int64(0)

In [694]:
df.loc[:, 'directed_by'].values.tolist()

[['D. W. Griffith'],
 ['Cecil B. DeMille'],
 ['.mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding:0}.mw-parser-output .plainlist ol li,.mw-parser-output .plainlist ul li{margin-bottom:0}',
  'Mike Todd',
  'Michael Todd, Jr.',
  'Walter A. Thompson',
  'Fred Rickey'],
 ['Robert L. Bendick', 'Philippe De Lacy'],
 ['Henry Koster'],
 ['Mervyn LeRoy'],
 ['Richard Fleischer'],
 ['Alfred Hitchcock'],
 ['.mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding:0}.mw-parser-output .plainlist ol li,.mw-parser-output .plainlist ul li{margin-bottom:0}',
  '\n',
  'Hamilton Luske',
  '\n',
  'Clyde Geronimi',
  '\n',
  'Wilfred Jackson',
  '\n'],
 ['Michael Curtiz'],
 ['.mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding:0}.mw-parser-output .plainlist ol li,.mw-parser-output .plainlist ul li{margin-bottom:0}',

In [695]:
# We need to remove unnecessary values from directed_by column

stop_words = ['\n', ' ', '(uncredited)', ' (uncredited) ', ' (uncredited)', '(uncredited) ', ' (action sequences)', ' (action sequences) ', '(action sequences) ',
                  '(action sequences)', ' (presented by) ', ' (presented by)', '(presented by) ', '(presented by)']

def remove_stop_words(words: list[str], stop_words: list[str], length_threshold: int = 49) -> list[str]:
    res = []
    #length_threshold = len('.mw-parser-output .plainlist ol,.mw-parser-output')
    for i in words:
        if not (any(i in stop_word for stop_word in stop_words) or len(i) >= length_threshold or len(i) <= 2):
            res.append(i)
    return res

df['directed_by'] = df.loc[:, 'directed_by'].apply(lambda x: remove_stop_words(x, stop_words))

In [696]:
df.head()

,title,directed_by,screenplay_by,based_on,produced_by,starring,release_date,country,budget,box_office
0,The Birth of a Nation,[D. W. Griffith],"[.mw-parser-output .plainlist ol,.mw-parser-ou...","[The Clansman, by , Thomas Dixon Jr.]","[D. W. Griffith, Harry Aitken, [, 1, ]]","[\n, Lillian Gish, \n, Mae Marsh, \n, Henry B....",1915,[United States],"[$100,000+, [, 3, ]]","[$50–100 million, [, 4, ]]"
1,The Greatest Show on Earth,[Cecil B. DeMille],"[.mw-parser-output .plainlist ol,.mw-parser-ou...",[],[Cecil B. DeMille],"[Betty Hutton, Cornel Wilde, Charlton Heston, ...",1916,[United States],"[$4 million, [, 1, ]]","[$36 million, [, 1, ]]"
2,This Is Cinerama,"[Mike Todd, Michael Todd, Jr., Walter A. Thomp...",[],[],"[Robert L. Bendick, Merian C. Cooper]",[Lowell Thomas],1917,[United States],[$1 million],"[$41.6 million, [, 2, ]]"
3,Cinerama Holiday,"[Robert L. Bendick, Philippe De Lacy]",[],[],[],"[Fred Troller, , Beatrice Troller , John Mars...",1918,[United States],"[$1.5 million, [, 1, ]]","[$29.6 million , [, 2, ]]"
4,The Robe,[Henry Koster],"[.mw-parser-output .plainlist ol,.mw-parser-ou...","[The Robe, by , Lloyd C. Douglas]",[Frank Ross],"[\n, Richard Burton, \n, Jean Simmons, \n, Vic...",1919,[United States],"[$4.1 million, [, 1, ], – $4.6 million, [, 2, ]]","[$36 million (United States), [, 3, ]]"


In [697]:
# Check the amount of empty elements in screenplay_by and based_on
df.loc[:, 'screenplay_by'].value_counts()[:3]

screenplay_by
[]                 52
[Steve Kloves]      3
[Ernest Lehman]     3
Name: count, dtype: int64

In [698]:
df.loc[:, 'based_on'].value_counts()[:3]

based_on
[]                                       46
[Avengers, by , Stan Lee, Jack Kirby]     3
[Characters, by , George Lucas]           2
Name: count, dtype: int64

Almost 33% of all entries in `screenplay_by` and `based_on` columns is empty. So, we can drop these columns.

In [699]:
df = df.drop(labels=["screenplay_by", "based_on"], axis=1)
df.head()

,title,directed_by,produced_by,starring,release_date,country,budget,box_office
0,The Birth of a Nation,[D. W. Griffith],"[D. W. Griffith, Harry Aitken, [, 1, ]]","[\n, Lillian Gish, \n, Mae Marsh, \n, Henry B....",1915,[United States],"[$100,000+, [, 3, ]]","[$50–100 million, [, 4, ]]"
1,The Greatest Show on Earth,[Cecil B. DeMille],[Cecil B. DeMille],"[Betty Hutton, Cornel Wilde, Charlton Heston, ...",1916,[United States],"[$4 million, [, 1, ]]","[$36 million, [, 1, ]]"
2,This Is Cinerama,"[Mike Todd, Michael Todd, Jr., Walter A. Thomp...","[Robert L. Bendick, Merian C. Cooper]",[Lowell Thomas],1917,[United States],[$1 million],"[$41.6 million, [, 2, ]]"
3,Cinerama Holiday,"[Robert L. Bendick, Philippe De Lacy]",[],"[Fred Troller, , Beatrice Troller , John Mars...",1918,[United States],"[$1.5 million, [, 1, ]]","[$29.6 million , [, 2, ]]"
4,The Robe,[Henry Koster],[Frank Ross],"[\n, Richard Burton, \n, Jean Simmons, \n, Vic...",1919,[United States],"[$4.1 million, [, 1, ], – $4.6 million, [, 2, ]]","[$36 million (United States), [, 3, ]]"


In [700]:
#Check the amount of empty entries in produced_by and starring columns
df.loc[:, 'produced_by'].value_counts()

produced_by
[Walt Disney]                    9
[Cecil B. DeMille]               5
[Sam Zimbalist]                  4
[Kevin Feige]                    4
[]                               2
                                ..
[Jesse L. Lasky]                 1
[George Loane Tucker]            1
[Rex Ingram]                     1
[Mabel Normand, Mack Sennett]    1
[D. W. Griffith]                 1
Name: count, Length: 117, dtype: int64

In [701]:
df.loc[:, 'starring'].value_counts()

starring
[]                                                                                                                                                                                                                                                        2
[\n, Mark Hamill, \n, Harrison Ford, \n, Carrie Fisher, \n, Billy Dee Williams, \n, Anthony Daniels, \n, David Prowse, \n, Kenny Baker, \n, Peter Mayhew, \n, Frank Oz, \n]                                                                               2
[\n, Sylvester Stallone, \n, Talia Shire, \n, Burt Young, \n, Carl Weathers, \n, Burgess Meredith, \n]                                                                                                                                                    2
[Fred Troller,  , Beatrice Troller , John Marsh, Betty Marsh]                                                                                                                                                                              

In [702]:
# Fill all empty entries with "Unknown" because we do not want to lose ary rows from our dataset
df['produced_by'] = df.loc[:, 'produced_by'].apply(lambda x: ["Unknown"] if len(x) == 0 else x)
df['starring'] = df.loc[:, 'starring'].apply(lambda x: ["Unknown"] if len(x) == 0 else x)

In [703]:
# Remove stopwords from both columns
stop_words = ['\n', ' ', '(uncredited)', ' (uncredited) ', ' (action sequences)', ' (action sequences) ', '(action sequences) ',
                  '(action sequences)', 'uncredited', 'Uncredited:']

df['starring'] = df.loc[:, 'starring'].apply(lambda x: remove_stop_words(x, stop_words))
df['produced_by'] = df.loc[:, 'produced_by'].apply(lambda x: remove_stop_words(x, stop_words))

In [704]:
# Remove stopwords from country column
df['country'] = df.loc[:, 'country'].apply(lambda x: remove_stop_words(x, stop_words))

In [705]:
df.loc[:, 'country'].value_counts()

country
[United States]     131
[]                   17
[United Kingdom]      1
[Japan]               1
Name: count, dtype: int64

In [706]:
# Fill empty country names as Unknown. We can save only the first element of each entry
# because we have only 1 country per film
df['country'] = df.loc[:, 'country'].apply(lambda x: "Unknown" if len(x) == 0 else x[0])

In [707]:
df.head()

,title,directed_by,produced_by,starring,release_date,country,budget,box_office
0,The Birth of a Nation,[D. W. Griffith],"[D. W. Griffith, Harry Aitken]","[Lillian Gish, Mae Marsh, Henry B. Walthall, M...",1915,United States,"[$100,000+, [, 3, ]]","[$50–100 million, [, 4, ]]"
1,The Greatest Show on Earth,[Cecil B. DeMille],[Cecil B. DeMille],"[Betty Hutton, Cornel Wilde, Charlton Heston, ...",1916,United States,"[$4 million, [, 1, ]]","[$36 million, [, 1, ]]"
2,This Is Cinerama,"[Mike Todd, Michael Todd, Jr., Walter A. Thomp...","[Robert L. Bendick, Merian C. Cooper]",[Lowell Thomas],1917,United States,[$1 million],"[$41.6 million, [, 2, ]]"
3,Cinerama Holiday,"[Robert L. Bendick, Philippe De Lacy]",[Unknown],"[Fred Troller, Beatrice Troller , John Marsh, ...",1918,United States,"[$1.5 million, [, 1, ]]","[$29.6 million , [, 2, ]]"
4,The Robe,[Henry Koster],[Frank Ross],"[Richard Burton, Jean Simmons, Victor Mature, ...",1919,United States,"[$4.1 million, [, 1, ], – $4.6 million, [, 2, ]]","[$36 million (United States), [, 3, ]]"


In [708]:
# Check if there are any empty values in budget and box_office
df.loc[:, 'budget'].value_counts()

budget
[$4 million, [, 1, ]]                4
[]                                   3
[$1.5 million, [, 1, ]]              2
[$1 million]                         2
[$3 million]                         2
                                    ..
[$250,000–500,000, [, 1, ]]          1
[$120,000]                           1
[$800,000 or $1 million, [, 1, ]]    1
[$250,000]                           1
[$385,907, [, 1, ]]                  1
Name: count, Length: 142, dtype: int64

In [709]:
df.loc[:, 'box_office'].value_counts()

box_office
[$50–100 million, [, 4, ]]                                                                                            1
[$36 million, [, 1, ]]                                                                                                1
[$41.6 million, [, 2, ]]                                                                                              1
[$29.6 million , [, 2, ]]                                                                                             1
[$36 million (United States), [, 3, ]]                                                                                1
                                                                                                                     ..
[$1 million, [, 1, ]]                                                                                                 1
[$1 million (rentals), [, 1, ],  or $2 million, [, 2, ]]                                                              1
[$9.2 million, [, 2, ],  or $

In [710]:
df.loc[:, 'budget'].tolist()

[['$100,000+', '[', '3', ']'],
 ['$4 million', '[', '1', ']'],
 ['$1 million'],
 ['$1.5 million', '[', '1', ']'],
 ['$4.1 million', '[', '1', ']', ' – $4.6 million', '[', '2', ']'],
 ['$7.6 million', '[', '1', ']'],
 ['$5 million', '[', '1', ']'],
 ['$1 million'],
 ['$4 million', '[', '1', ']'],
 ['$2 million', '[', '2', ']'],
 ['$4 million', '[', '1', ']'],
 ['$2.3 million', '[', '2', ']'],
 ['$3.8 million', '[', '1', ']'],
 ['$2.9', '[', '2', ']', '–3.1 million', '[', '3', ']'],
 ['$200\xa0million', '[', '2', ']'],
 ['> ', '£', '505,600', '[', 'a', ']', '[', '1', ']', '[', '2', ']'],
 ['$2.2 million', '[', '1', ']'],
 ['$350–460\xa0million', '[', '2', ']', '[', '3', ']'],
 ['$15.7 million', '[', '1', ']'],
 ['$128–145\xa0million', '[', '4', ']', '[', '5', ']'],
 ['$200', '\xa0', 'million', '[', '2', ']'],
 ['$300 million', '[', '2', ']'],
 ['$325–400 million', '[', '2', ']', '[', '3', ']'],
 ['$356–400', '\xa0', 'million', '[', '2', ']', '[', '3', ']'],
 ['$210 million', '[', '4', ']

In [711]:
# Remove stopwords from budget column
stop_words = ['\n', ' ', '(uncredited)', ' (uncredited) ', ' (action sequences)', ' (action sequences) ', '(action sequences) ',
                  '(action sequences)', 'uncredited', 'Uncredited:', '(est.)', '(estimated)']
df['budget'] = df.loc[:, 'budget'].apply(lambda x: remove_stop_words(x, stop_words)).tolist()

In [712]:
# Fill empty values in the budget column
df['budget'] = df.loc[:, 'budget'].apply(lambda x: ['$0.0'] if len(x) == 0 else x)

In [713]:
def delete_wrong_symbol(value: list[str]) -> list[int, int]:
    return [" ".join(value).replace("\xa0", " ")]

df['budget'] = df.loc[:, 'budget'].apply(delete_wrong_symbol).tolist()

In [714]:
pattern = re.compile(r'\$(\d+(?:[.,]\d{3})*(?:\.\d+)?)\s*million')

def scale_number(value):
    # Replace commas with dots for consistency
    value = value.replace(',', '.')
    # Convert to float and multiply by 1,000,000
    return int(float(value) * 1_000_000)

def parse_numbers(value: list[str], pattern: re.Pattern = pattern):
    matches = pattern.findall(value[0])
    if matches:
        #scaled_values = [scale_number(match) for match in matches]
        #print(f"Input: {value[0]} -> {scaled_values}")
        return matches

In [715]:
df.loc[:, 'budget'].to_list()

[['$100,000+'],
 ['$4 million'],
 ['$1 million'],
 ['$1.5 million'],
 ['$4.1 million  – $4.6 million'],
 ['$7.6 million'],
 ['$5 million'],
 ['$1 million'],
 ['$4 million'],
 ['$2 million'],
 ['$4 million'],
 ['$2.3 million'],
 ['$3.8 million'],
 ['$2.9 –3.1 million'],
 ['$200 million'],
 ['505,600'],
 ['$2.2 million'],
 ['$350–460 million'],
 ['$15.7 million'],
 ['$128–145 million'],
 ['$200 million'],
 ['$300 million'],
 ['$325–400 million'],
 ['$356–400 million'],
 ['$210 million'],
 ['$447 million (net) $533 million (gross)'],
 ['$250 million'],
 ['$150 million'],
 ['$200 million'],
 ['$250 million (shared with  Part 1'],
 ['$220–225 million'],
 ['$150 million'],
 ['$300 million'],
 ['$225 million'],
 ['$150 million'],
 ['$94 million'],
 ['$185 million'],
 ['$237 million'],
 ['$120–125 million'],
 ['$140 million'],
 ['$94 million'],
 ['$125 million'],
 ['$115 million'],
 ['$200 million'],
 ['$90 million'],
 ['$75 million'],
 ['$30 million'],
 ['$45 million'],
 ['$22–23 million'],
 

In [716]:
pattern = re.compile(r'\d+(?:[.,])*\d*(?:[.,]*)\d*')
df['budget'] = df.loc[:, 'budget'].apply(lambda x: parse_numbers(x, pattern)).tolist()

In [717]:
df.loc[:, 'budget'].tolist()

[['100,000'],
 ['4'],
 ['1'],
 ['1.5'],
 ['4.1', '4.6'],
 ['7.6'],
 ['5'],
 ['1'],
 ['4'],
 ['2'],
 ['4'],
 ['2.3'],
 ['3.8'],
 ['2.9', '3.1'],
 ['200'],
 ['505,600'],
 ['2.2'],
 ['350', '460'],
 ['15.7'],
 ['128', '145'],
 ['200'],
 ['300'],
 ['325', '400'],
 ['356', '400'],
 ['210'],
 ['447', '533'],
 ['250'],
 ['150'],
 ['200'],
 ['250', '1'],
 ['220', '225'],
 ['150'],
 ['300'],
 ['225'],
 ['150'],
 ['94'],
 ['185'],
 ['237'],
 ['120', '125'],
 ['140'],
 ['94'],
 ['125'],
 ['115'],
 ['200'],
 ['90'],
 ['75'],
 ['30'],
 ['45'],
 ['22', '23'],
 ['14'],
 ['25'],
 ['28'],
 ['48'],
 ['94', '102'],
 ['63'],
 ['15'],
 ['32.5', '42.7'],
 ['25', '30'],
 ['19'],
 ['7'],
 ['10.5'],
 ['6'],
 ['34'],
 ['30.5'],
 ['20'],
 ['14'],
 ['5.5'],
 ['1,100,000'],
 ['11'],
 ['12'],
 ['9'],
 ['9'],
 ['1.8', '2.2'],
 ['7.2'],
 ['2.2'],
 ['6', '7', '1'],
 ['14.1'],
 ['6'],
 ['15'],
 ['3'],
 ['4'],
 ['7.5'],
 ['10.5'],
 ['15', '18'],
 ['2'],
 ['4.4', '6'],
 ['8.2'],
 ['3'],
 ['17'],
 ['31.1'],
 ['5'],
 ['6.7

In [718]:
def process_number(num: str) -> int:
    num = num.replace(",", ".")
    num_splitted = num.split(".")

    if "." not in num:
        return int(num) * 1_000_000
    elif len(num_splitted[0]) > 2 and len(num_splitted[1]) > 2:
        return int(num.replace(".", "_"))
    elif len(num_splitted) >= 3:
        return int(num_splitted[0]) * 1_000_000 + int(num_splitted[1]) * 1_000 + int(num_splitted[2])
    else:
        return int(num_splitted[0]) * 1_000_000 + int("".join(num_splitted[1:])) * 1_000

def process_number_list(nums: list[str]) -> tuple[int, int]:
    if len(nums) == 1:
        return process_number(nums[0])
    
    numbers = []
    for idx, num in enumerate(nums):
        if idx == 2:
            break
        numbers.append(process_number(num))
    
    return numbers

In [719]:
df['budget'] = df.loc[:, 'budget'].apply(process_number_list).tolist()

In [720]:
df.head()

,title,directed_by,produced_by,starring,release_date,country,budget,box_office
0,The Birth of a Nation,[D. W. Griffith],"[D. W. Griffith, Harry Aitken]","[Lillian Gish, Mae Marsh, Henry B. Walthall, M...",1915,United States,100000,"[$50–100 million, [, 4, ]]"
1,The Greatest Show on Earth,[Cecil B. DeMille],[Cecil B. DeMille],"[Betty Hutton, Cornel Wilde, Charlton Heston, ...",1916,United States,4000000,"[$36 million, [, 1, ]]"
2,This Is Cinerama,"[Mike Todd, Michael Todd, Jr., Walter A. Thomp...","[Robert L. Bendick, Merian C. Cooper]",[Lowell Thomas],1917,United States,1000000,"[$41.6 million, [, 2, ]]"
3,Cinerama Holiday,"[Robert L. Bendick, Philippe De Lacy]",[Unknown],"[Fred Troller, Beatrice Troller , John Marsh, ...",1918,United States,1005000,"[$29.6 million , [, 2, ]]"
4,The Robe,[Henry Koster],[Frank Ross],"[Richard Burton, Jean Simmons, Victor Mature, ...",1919,United States,"[4001000, 4006000]","[$36 million (United States), [, 3, ]]"


In [721]:
df.loc[:, 'box_office'].tolist()

[['$50–100 million', '[', '4', ']'],
 ['$36 million', '[', '1', ']'],
 ['$41.6 million', '[', '2', ']'],
 ['$29.6 million ', '[', '2', ']'],
 ['$36 million (United States)', '[', '3', ']'],
 ['$21 million'],
 ['$28.2 million', '[', '2', ']'],
 ['$37.9 million', '[', '3', ']'],
 ['$187 million', '[', '2', ']'],
 ['$30 million', '[', '3', ']'],
 ['$87.4 million (United States and Canada)', '[', '1', ']'],
 ['$15.1 million', '[', '3', ']'],
 ['$10 million (Worldwide) ', '[', '2', ']', '[', '3', ']'],
 ['$25.6 million', '[', '4', ']'],
 ['$1.699\xa0billion', '[', '3', ']', '[', '4', ']'],
 ['$5 million (U.S. and Canada rentals)', '[', '3', ']'],
 ['$182 million', '[', '2', ']'],
 ['$2.320\xa0billion', '[', '4', ']', '[', '5', ']'],
 ['$507.1 million', '[', '1', ']'],
 ['$1.446\xa0billion', '[', '6', ']', '[', '7', ']'],
 ['$1.923 billion', '[', '3', ']', '[', '4', ']'],
 ['$1.334', '\xa0', 'billion', '[', '3', ']'],
 ['$2.052\xa0billion', '[', '4', ']'],
 ['$2.799', '\xa0', 'billion', '[',

In [722]:
# Process box_office column in the same way
df['box_office'] = df.loc[:, 'box_office'].apply(lambda x: remove_stop_words(x, stop_words)).tolist()
df['box_office'] = df.loc[:, 'box_office'].apply(lambda x: ['$0.0'] if len(x) == 0 else x)
df['box_office'] = df.loc[:, 'box_office'].apply(delete_wrong_symbol)

pattern = re.compile(r'\d+(?:[.,])*\d*(?:[.,]*)\d*')
#df.loc[:, 'box_office'].apply(lambda x: parse_numbers(x, pattern)).tolist()
df.loc[:, 'box_office'].tolist()

[['$50–100 million'],
 ['$36 million'],
 ['$41.6 million'],
 ['$29.6 million '],
 ['$36 million (United States)'],
 ['$21 million'],
 ['$28.2 million'],
 ['$37.9 million'],
 ['$187 million'],
 ['$30 million'],
 ['$87.4 million (United States and Canada)'],
 ['$15.1 million'],
 ['$10 million (Worldwide) '],
 ['$25.6 million'],
 ['$1.699 billion'],
 ['$5 million (U.S. and Canada rentals)'],
 ['$182 million'],
 ['$2.320 billion'],
 ['$507.1 million'],
 ['$1.446 billion'],
 ['$1.923 billion'],
 ['$1.334 billion'],
 ['$2.052 billion'],
 ['$2.799 billion'],
 ['$1.104 billion'],
 ['$2.07 billion'],
 ['$1.155 billion'],
 ['$1.280 billion'],
 ['$1.067 billion'],
 ['$1.342 billion'],
 ['$1.521 billion'],
 ['$935.5 million'],
 ['$963.4 million'],
 ['$1.066 billion'],
 ['$897.5 million'],
 ['$1.151 billion'],
 ['$1.006 billion'],
 ['$2.923 billion'],
 ['$546.3 million'],
 ['$553.7 million'],
 ['$951.6 million'],
 ['$1.026 billion'],
 ['$1.047 billion'],
 ['$2.264 billion'],
 ['$366.1 million'],
 [

In [723]:
def scale_number(value, suffix):
    value = value.replace(',', '.')
    numeric_value = float(value)
    if suffix == 'million':
        return int(numeric_value * 1_000_000)
    elif suffix == 'billion':
        return int(numeric_value * 1_000_000_000)
    else:
        return int(numeric_value)

def extract_and_scale_money_values(item):
    if isinstance(item, list):
        item = item[0]

    pattern = re.compile(r'\$(\d+(?:[.,]\d{3})*(?:\.\d+)?)\s*(million|billion)?')

    def scale_number(value, suffix):
        num = value.replace(',', '.')
        #num = float(value)
        num_splitted = num.split(".")
        if suffix == 'million':
            if "." not in num:
                return int(num) * 1_000_000
            elif len(num_splitted[0]) > 2 and len(num_splitted[1]) > 2:
                return int(num.replace(".", "_"))
            elif len(num_splitted) >= 3:
                return int(num_splitted[0]) * 1_000_000 + int(num_splitted[1]) * 1_000 + int(num_splitted[2])
            else:
                return int(num_splitted[0]) * 1_000_000 + int("".join(num_splitted[1:])) * 1_000
        elif suffix == 'billion':
            if "." not in num:
                return int(num) * 1_000_000_000
            elif len(num_splitted[0]) > 2 and len(num_splitted[1]) > 2:
                return int(num.replace(".", "_"))
            elif len(num_splitted) >= 3:
                return int(num_splitted[0]) * 1_000_000_000 + int(num_splitted[1]) * 1_000_000 + int(num_splitted[2]) * 1_000
            else:
                return int(num_splitted[0]) * 1_000_000_000 + int("".join(num_splitted[1:])) * 1_000_000
        else:
            return int(num.replace(".", "_"))

    matches = pattern.findall(item)
    scaled_values = []
    for match in matches:
        value, suffix = match
        #print(value, suffix)
        scaled_values.append(scale_number(value, suffix))

    if len(scaled_values) == 1:
        return scaled_values[0]
    else:
        return scaled_values
    
df['box_office'] = df.loc[:, 'box_office'].apply(extract_and_scale_money_values).tolist()

In [724]:
df['box_office'] = df.loc[:, 'box_office'].apply(lambda x: x if len(str(x)) >= 4 else x * 1_000_000)

In [725]:
df.head()

,title,directed_by,produced_by,starring,release_date,country,budget,box_office
0,The Birth of a Nation,[D. W. Griffith],"[D. W. Griffith, Harry Aitken]","[Lillian Gish, Mae Marsh, Henry B. Walthall, M...",1915,United States,100000,50000000
1,The Greatest Show on Earth,[Cecil B. DeMille],[Cecil B. DeMille],"[Betty Hutton, Cornel Wilde, Charlton Heston, ...",1916,United States,4000000,36000000
2,This Is Cinerama,"[Mike Todd, Michael Todd, Jr., Walter A. Thomp...","[Robert L. Bendick, Merian C. Cooper]",[Lowell Thomas],1917,United States,1000000,41006000
3,Cinerama Holiday,"[Robert L. Bendick, Philippe De Lacy]",[Unknown],"[Fred Troller, Beatrice Troller , John Marsh, ...",1918,United States,1005000,29006000
4,The Robe,[Henry Koster],[Frank Ross],"[Richard Burton, Jean Simmons, Victor Mature, ...",1919,United States,"[4001000, 4006000]",36000000


In [726]:
# Save our preprocessed dataframe into a csv file
df.to_csv("data/df_preprocessed.csv")